# 超参数

- 超参数：算法运行前需要决定的参数
- 模型参数：算法过程中学习的参数

在KNN算法中, k 就是超参数

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

digits = datasets.load_digits()
X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

KNN_classifier = KNeighborsClassifier(n_neighbors=3)
KNN_classifier.fit(X_train, y_train)
KNN_classifier.score(X_test, y_test)

0.98888888888888893

## 寻找最好的K

In [3]:
best_score = 0
best_k = -1
for k in range(1, 11):
    knn_clf = KNeighborsClassifier(n_neighbors=k)
    knn_clf.fit(X_train, y_train)
    score = knn_clf.score(X_test, y_test)
    if score > best_score:
        best_k = k
        best_score = score

print("best_k = ", best_k)
print("best_score = ", best_score)

best_k =  4
best_score =  0.991666666667


如果best_k=10, 则有可能更好的k值更大，需要增加k的范围重新获取最好的k,如返回设置为range(8, 20)

## 考虑距离？不考虑距离

In [5]:
best_method = ""
best_score = 0
best_k = -1
for method in ["uniform", "distance"]:
    for k in range(1, 11):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights=method)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k = k
            best_score = score
            best_method = method

print("best_k = ", best_k)
print("best_score = ", best_score)
print("best_method = ", best_method)

best_k =  4
best_score =  0.991666666667
best_method =  uniform


- 欧拉距离
- 曼哈顿距离
- 明科夫斯基距离

## 探索明科夫斯基距离相应的超参数p

In [7]:
%%time
best_p = -1
best_score = 0
best_k = -1

for k in range(1, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights='distance', p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k = k
            best_score = score
            best_p = p

print("best_k = ", best_k)
print("best_score = ", best_score)
print("best_p = ", best_p)

best_k =  3
best_score =  0.988888888889
best_p =  2
CPU times: user 17.2 s, sys: 117 ms, total: 17.3 s
Wall time: 17.7 s


## 网格搜索
存在多个超参数时，网格搜索即是使用穷举法来将所用的参数组合都运行一遍

In [20]:
param_grid = [
    {
        'weights': ['uniform'],
        'n_neighbors': [i for i in range(1, 11)]
    },
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(1, 11)],
        'p': [i for i in range(1, 6)]
    }
]


In [21]:
knn_clf = KNeighborsClassifier()

In [22]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(knn_clf, param_grid)

In [23]:
%%time
grid_search.fit(X_train, y_train)

CPU times: user 2min 18s, sys: 875 ms, total: 2min 19s
Wall time: 2min 25s


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [24]:
# 查看最近参数
grid_search.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=3,
           weights='distance')

In [25]:
grid_search.best_score_

0.98538622129436326

In [26]:
grid_search.best_params_

{'n_neighbors': 3, 'p': 3, 'weights': 'distance'}

In [27]:
knn_clf = grid_search.best_estimator_

In [28]:
knn_clf.score(X_test, y_test)

0.98333333333333328

In [29]:
%%time
# n_jobs 多核处理
# verbose 输出的信息
grid_search = GridSearchCV(knn_clf, param_grid, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV] n_neighbors=1, weights=uniform ..................................
[CV] n_neighbors=1, weights=uniform ..................................
[CV] n_neighbors=1, weights=uniform ..................................
[CV] n_neighbors=2, weights=uniform ..................................
[CV] ................... n_neighbors=1, weights=uniform, total=   0.8s
[CV] n_neighbors=2, weights=uniform ..................................
[CV] ................... n_neighbors=1, weights=uniform, total=   0.9s
[CV] n_neighbors=2, weights=uniform ..................................
[CV] ................... n_neighbors=1, weights=uniform, total=   0.9s
[CV] n_neighbors=3, weights=uniform ..................................
[CV] ................... n_neighbors=2, weights=uniform, total=   1.0s
[CV] n_neighbors=3, weights=uniform ..................................
[CV] ................... n_neighbors=2, weights=uniform, total=   1.0s
[CV] n_neighbor

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   20.6s


[CV] .................. n_neighbors=10, weights=uniform, total=   1.1s
[CV] n_neighbors=1, p=3, weights=distance ............................
[CV] ............. n_neighbors=1, p=3, weights=distance, total=   0.9s
[CV] n_neighbors=1, p=3, weights=distance ............................
[CV] .................. n_neighbors=10, weights=uniform, total=   1.1s
[CV] n_neighbors=1, p=4, weights=distance ............................
[CV] ............. n_neighbors=1, p=3, weights=distance, total=   1.0s
[CV] n_neighbors=1, p=4, weights=distance ............................
[CV] .................. n_neighbors=10, weights=uniform, total=   1.2s
[CV] n_neighbors=1, p=4, weights=distance ............................
[CV] ............. n_neighbors=1, p=3, weights=distance, total=   0.9s
[CV] n_neighbors=1, p=5, weights=distance ............................
[CV] ............. n_neighbors=1, p=4, weights=distance, total=   1.1s
[CV] n_neighbors=1, p=5, weights=distance ............................
[CV] .

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.3min


[CV] ............. n_neighbors=8, p=5, weights=distance, total=   1.0s
[CV] n_neighbors=9, p=3, weights=distance ............................
[CV] ............. n_neighbors=8, p=5, weights=distance, total=   1.3s
[CV] n_neighbors=9, p=4, weights=distance ............................
[CV] ............. n_neighbors=9, p=3, weights=distance, total=   1.2s
[CV] n_neighbors=9, p=4, weights=distance ............................
[CV] ............. n_neighbors=9, p=3, weights=distance, total=   1.2s
[CV] n_neighbors=9, p=4, weights=distance ............................
[CV] ............. n_neighbors=9, p=3, weights=distance, total=   1.1s
[CV] n_neighbors=9, p=5, weights=distance ............................
[CV] ............. n_neighbors=9, p=4, weights=distance, total=   1.0s
[CV] n_neighbors=9, p=5, weights=distance ............................
[CV] ............. n_neighbors=9, p=4, weights=distance, total=   1.0s
[CV] n_neighbors=9, p=5, weights=distance ............................
[CV] .

[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.5min finished


### 更多的距离定义

- 向量空间余弦相似度
- 调整余弦相似度
- 皮尔森相关系数
- Jaccard相似系数